In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [17]:
import gzip
import numpy as np

def read_idx_images(filepath):
    with gzip.open(filepath, 'rb') as f:
        _ = int.from_bytes(f.read(4), 'big')
        num_images = int.from_bytes(f.read(4), 'big')
        rows = int.from_bytes(f.read(4), 'big')
        cols = int.from_bytes(f.read(4), 'big')
        data = np.frombuffer(f.read(), dtype=np.uint8)
        return data.reshape(num_images, rows * cols) / 255.0  # Normalize to [0,1]

def read_idx_labels(filepath):
    with gzip.open(filepath, 'rb') as f:
        _ = int.from_bytes(f.read(4), 'big')
        num_labels = int.from_bytes(f.read(4), 'big')
        data = np.frombuffer(f.read(), dtype=np.uint8)
        return data


In [19]:
NUM_CLASSES = 26
TRAIN_SAMPLES_PER_CLASS = 200
TEST_SAMPLES_PER_CLASS = 20

def split_data_by_class(images, labels, samples_per_class, num_classes=26):
    data_by_class = {}
    for cls in range(1, num_classes + 1):  # کلاس‌ها از 1 تا 26
        indices = np.where(labels == cls)[0][:samples_per_class]
        data_by_class[cls - 1] = images[indices].T
    return data_by_class


In [20]:
train_images_path = "/content/drive/MyDrive/emnist-letters-train-images-idx3-ubyte.gz"
train_labels_path = "/content/drive/MyDrive/emnist-letters-train-labels-idx1-ubyte.gz"
test_images_path = "/content/drive/MyDrive/emnist-letters-test-images-idx3-ubyte.gz"
test_labels_path = "/content/drive/MyDrive/emnist-letters-test-labels-idx1-ubyte.gz"


train_images = read_idx_images(train_images_path)
train_labels = read_idx_labels(train_labels_path)
test_images = read_idx_images(test_images_path)
test_labels = read_idx_labels(test_labels_path)


train_data = split_data_by_class(train_images, train_labels, TRAIN_SAMPLES_PER_CLASS)
test_data = split_data_by_class(test_images, test_labels, TEST_SAMPLES_PER_CLASS)


test_labels_filtered = np.array([cls for cls in range(NUM_CLASSES) for _ in range(TEST_SAMPLES_PER_CLASS)])


In [21]:
def householder_qr(A):
    """
    Performs QR decomposition of matrix A using Householder reflections.
    A is of shape (m, n). Returns Q (m x m), R (m x n).
    """
    A = A.copy()
    m, n = A.shape
    Q = np.eye(m)

    for k in range(n):
        x = A[k:, k]
        e = np.zeros_like(x)
        e[0] = np.linalg.norm(x) * np.sign(x[0] if x[0] != 0 else 1)

        u = x - e
        if np.linalg.norm(u) == 0:
            continue
        v = u / np.linalg.norm(u)

        H_k = np.eye(m)
        H_k[k:, k:] -= 2.0 * np.outer(v, v)

        A = H_k @ A
        Q = Q @ H_k.T
    R = A
    return Q, R


In [22]:
def solve_least_squares(Q, R, b):
    """
    حل کمترین مربعات با QR: Ax ≈ b → R x = Q^T b
    """
    Qt_b = Q.T @ b

    x = np.zeros(R.shape[1])
    for i in reversed(range(R.shape[1])):
        x[i] = (Qt_b[i] - np.dot(R[i, i+1:], x[i+1:])) / R[i, i]
    return x


In [23]:
def predict_sample(test_sample, train_data, qr_cache):
    errors = []
    for cls_idx in range(NUM_CLASSES):
        A = train_data[cls_idx]
        Q, R = qr_cache[cls_idx]
        x = solve_least_squares(Q, R, test_sample)
        recon = A @ x
        error = np.linalg.norm(recon - test_sample)
        errors.append((error, cls_idx))

    return min(errors, key=lambda e: e[0])[1]


In [ ]:
from sklearn.metrics import classification_report

def evaluate_model_with_report(test_data, train_data, qr_cache):
    y_true = []
    y_pred = []
    for cls_idx in range(NUM_CLASSES):
        for j in range(TEST_SAMPLES_PER_CLASS):
            test_sample = test_data[cls_idx][:, j]
            pred_label = predict_sample(test_sample, train_data, qr_cache)
            y_true.append(cls_idx)
            y_pred.append(pred_label)

    accuracy = np.mean(np.array(y_true) == np.array(y_pred))
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, digits=4))
    return accuracy


In [24]:
qr_cache = {}
for cls_idx in range(NUM_CLASSES):
    Q, R = householder_qr(train_data[cls_idx])
    qr_cache[cls_idx] = (Q, R)


In [26]:
accuracy = evaluate_model_with_report(test_data, train_data, qr_cache)


Accuracy: 70.58%

Classification Report:
              precision    recall  f1-score   support

           0     0.4839    0.7500    0.5882        20
           1     0.7778    0.7000    0.7368        20
           2     0.8125    0.6500    0.7222        20
           3     0.6154    0.4000    0.4848        20
           4     0.7143    0.7500    0.7317        20
           5     1.0000    0.5000    0.6667        20
           6     0.6111    0.5500    0.5789        20
           7     0.8667    0.6500    0.7429        20
           8     0.3778    0.8500    0.5231        20
           9     0.9231    0.6000    0.7273        20
          10     0.9474    0.9000    0.9231        20
          11     0.7500    0.3000    0.4286        20
          12     0.8500    0.8500    0.8500        20
          13     0.5172    0.7500    0.6122        20
          14     0.6071    0.8500    0.7083        20
          15     0.7619    0.8000    0.7805        20
          16     0.6000    0.4500    0.5

In [27]:
def givens_rotation(a, b):
    """
    Returns cosine and sine values for the Givens rotation that zeros out b.
    """
    if b == 0:
        return 1.0, 0.0
    r = np.hypot(a, b)
    c = a / r
    s = -b / r
    return c, s

def apply_givens_to_columns(R, c, s, i, k):
    """
    Applies a Givens rotation to rows i and k of matrix R (from the left).
    """
    G = np.array([[c, -s], [s, c]])
    R[[i, k], :] = G @ R[[i, k], :]

def update_qr_multiple_columns_givens_verbose(Q, R, X_new):
    """
    Updates the QR decomposition (Q, R) with new columns in X_new using Givens rotations.
    Prints progress during each column update.
    """
    m = Q.shape[0]
    R_aug = np.hstack((R, Q.T @ X_new))
    Q_total = np.eye(m)

    n_existing = R.shape[1]
    n_new = X_new.shape[1]

    for j in range(n_existing, n_existing + n_new):
        for i in range(m - 1, j, -1):
            a = R_aug[i - 1, j]
            b = R_aug[i, j]
            r = np.hypot(a, b)
            if r == 0:
                continue
            c = a / r
            s = -b / r

            G = np.eye(m)
            G[[i - 1, i], [i - 1, i]] = c
            G[i - 1, i] = -s
            G[i, i - 1] = s

            R_aug = G @ R_aug
            Q_total = Q_total @ G.T

    Q_new = Q @ Q_total
    R_new = R_aug
    return Q_new, R_new



qr_cache_updated = {}
for cls_idx in range(NUM_CLASSES):
    print(f"\n Updating QR for class {cls_idx}...")


    X_new = train_images[train_labels == cls_idx + 1][TRAIN_SAMPLES_PER_CLASS:TRAIN_SAMPLES_PER_CLASS + 20].T

    Q_old, R_old = qr_cache[cls_idx]
    Q_updated, R_updated = update_qr_multiple_columns_givens_verbose(Q_old, R_old, X_new)


    train_data[cls_idx] = np.hstack((train_data[cls_idx], X_new))
    qr_cache_updated[cls_idx] = (Q_updated, R_updated)

print("\n All QR updates complete")
accuracy_updated = evaluate_model_with_report(test_data, train_data, qr_cache_updated)



 Updating QR for class 0...

 Updating QR for class 1...

 Updating QR for class 2...

 Updating QR for class 3...

 Updating QR for class 4...

 Updating QR for class 5...

 Updating QR for class 6...

 Updating QR for class 7...

 Updating QR for class 8...

 Updating QR for class 9...

 Updating QR for class 10...

 Updating QR for class 11...

 Updating QR for class 12...

 Updating QR for class 13...

 Updating QR for class 14...

 Updating QR for class 15...

 Updating QR for class 16...

 Updating QR for class 17...

 Updating QR for class 18...

 Updating QR for class 19...

 Updating QR for class 20...

 Updating QR for class 21...

 Updating QR for class 22...

 Updating QR for class 23...

 Updating QR for class 24...

 Updating QR for class 25...

 All QR updates complete
Accuracy: 68.46%

Classification Report:
              precision    recall  f1-score   support

           0     0.5556    0.7500    0.6383        20
           1     0.7222    0.6500    0.6842        20
